In [1]:
from dash import Dash, html, dcc, Input, Output, callback
import pandas as pd
import plotly.express as px
import numpy as np

In [2]:
df=pd.read_csv('world_happiness_2021.csv')
df

,Country name,Regional indicator,Ladder score,Standard error of ladder score,upperwhisker,lowerwhisker,GDP per capita,Social support,Healthy life expectancy,Freedom to make life choices,Generosity,Perceptions of corruption,Ladder score in Dystopia,Explained by: Log GDP per capita,Explained by: Social support,Explained by: Healthy life expectancy,Explained by: Freedom to make life choices,Explained by: Generosity,Explained by: Perceptions of corruption,Dystopia + residual
0,Finland,Western Europe,7.842,0.032,7.904,7.780,10.775,0.954,72.000,0.949,-0.098,0.186,2.43,1.446,1.106,0.741,0.691,0.124,0.481,3.253
1,Denmark,Western Europe,7.620,0.035,7.687,7.552,10.933,0.954,72.700,0.946,0.030,0.179,2.43,1.502,1.108,0.763,0.686,0.208,0.485,2.868
2,Switzerland,Western Europe,7.571,0.036,7.643,7.500,11.117,0.942,74.400,0.919,0.025,0.292,2.43,1.566,1.079,0.816,0.653,0.204,0.413,2.839
3,Iceland,Western Europe,7.554,0.059,7.670,7.438,10.878,0.983,73.000,0.955,0.160,0.673,2.43,1.482,1.172,0.772,0.698,0.293,0.170,2.967
4,Netherlands,Western Europe,7.464,0.027,7.518,7.410,10.932,0.942,72.400,0.913,0.175,0.338,2.43,1.501,1.079,0.753,0.647,0.302,0.384,2.798
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
144,Lesotho,Sub-Saharan Africa,3.512,0.120,3.748,3.276,7.926,0.787,48.700,0.715,-0.131,0.915,2.43,0.451,0.731,0.007,0.405,0.103,0.015,1.800
145,Botswana,Sub-Saharan Africa,3.467,0.074,3.611,3.322,9.782,0.784,59.269,0.824,-0.246,0.801,2.43,1.099,0.724,0.340,0.539,0.027,0.088,0.648
146,Rwanda,Sub-Saharan Africa,3.415,0.068,3.548,3.282,7.676,0.552,61.400,0.897,0.061,0.167,2.43,0.364,0.202,0.407,0.627,0.227,0.493,1.095
147,Zimbabwe,Sub-Saharan Africa,3.145,0.058,3.259,3.030,7.943,0.750,56.201,0.677,-0.047,0.821,2.43,0.457,0.649,0.243,0.359,0.157,0.075,1.205


In [3]:
from dash import Dash, html, dcc, Input, Output, ctx, callback,dash_table, State
import pandas as pd
import plotly.express as px

marked=np.array([])
df1=pd.DataFrame(data={'Country':marked})

external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = Dash(__name__, external_stylesheets=external_stylesheets)

app.layout =html.Div([
     html.Div([

    dcc.Dropdown(
                 df.columns[2:],
                value='Ladder score',
                id='scatter3',
            )
    ] , style={'width': '20%', 'display': 'inline-block','padding':'0px 0px 0px 100px'}),
    
    html.Div([

    dcc.Dropdown(
                df['Regional indicator'].unique(),multi=True,
                value=['Western Europe','East Asia'],
                id='Regional indicator',
            )
    ] , style={'width': '20%', 'display': 'inline-block','padding':'0px 400px 0px 0px'}),

    
    html.Div([

    dcc.Dropdown(
                df.columns[2:],
                value='Ladder score',
                id='scatter1',
            )
    ] , style={'width': '10%', 'display': 'inline-block'}),

    html.Div([
    dcc.Dropdown(
                df.columns[2:],
                value='Social support',
                id='scatter2',
            )
    ] , style={'width': '10%', 'display': 'inline-block'}),
    
  

    
html.Br(),
html.Br(),   

    html.Div([
        dcc.Graph(
            id='Regional indicator-box',style={'width': '100vh', 'height': '45vh'}
           
        )
    ], style={'width': '45%', 'padding': '0px 300px 0px 0px', 'display': 'inline-block' } ),

    html.Div([
        dcc.Graph(
            id='scatter-plot',style={'width': '90vh', 'height': '45vh'},
            hoverData={'points': [{'customdata': 'Finland'}]}#initialise hoverdata update variable 'Japan' later
        )
    ], style={'width': '45%', 'padding': '0 20', 'display': 'inline-block' } ),
html.Br(),
html.Br(),   
  html.Div(dcc.Dropdown(
       np.unique(df['Country name']),
        id='countries',
    ),style={'fontSize':20,'textAlign': 'center','width': '12%', 'float': 'left', 'display': 'inline-block'}),

html.Button('Add Country', id='mark',n_clicks=0, style={'font-size': '20px'}),
html.Button('Remove', id='Remove3',n_clicks=0, style={'font-size': '20px'}),    
html.Button('Remove Last Country hovered', id='Remove',n_clicks=0, style={'font-size': '20px'}),    
html.Button('Remove Last Row', id='Remove2',n_clicks=0, style={'font-size': '20px'}),  
    
html.Br(),
html.Br(),   

    
     html.Div(dash_table.DataTable(id='Countries_table',                                  
                sort_action="native",page_action="native",                         
                style_header={  'textAlign': 'center'},
    style_cell={'fontSize':25,
        'textAlign': 'center', 'font-family':'calibri'
    }), style={'width': '50%', 'float': 'centre', 'padding': '0px 20px 20px 20px'}),
    dcc.Store(
       
        id='Countries_data',data=df1.to_dict(orient='records')
    ) 


])

@callback(
    Output('Regional indicator-box', 'figure'),
    Input('Regional indicator', 'value'),
Input('scatter3', 'value') )

def update_graph(Regional_indicator,scatter3):
    
    dff = df[df['Regional indicator'].isin(Regional_indicator)]
    
    fig = px.box(dff, y=scatter3,color='Regional indicator')
    
    fig.update_yaxes(title=scatter3,titlefont=dict(size=25),tickfont=dict(size=25))
  

    return fig   



@callback(
    Output('scatter-plot', 'figure'),
    Input('Regional indicator', 'value'),
    Input('scatter1', 'value'),
    Input('scatter2', 'value'))
def update_graph(Regional_indicator, scatter1,
                 scatter2):
    dff = df[df['Regional indicator'].isin(Regional_indicator)]
    

    fig = px.scatter(dff,x=dff[scatter2],y=dff[scatter1],hover_name=dff['Country name'],color='Regional indicator')#color messes up hoverdata for some reason
    
    fig.update_traces(customdata=dff['Country name'])
    fig.update_xaxes(title=scatter2,titlefont=dict(size=25),tickfont=dict(size=25))
  
    fig.update_yaxes(title=scatter1,titlefont=dict(size=25),tickfont=dict(size=25))

    fig.update_layout(margin={'l': 40, 'b': 40, 't': 10, 'r': 0}, hovermode='closest')

    return fig    


@callback(
             Output('Countries_data', 'data'),
             Input('scatter-plot', 'hoverData'),
             Input('Regional indicator', 'value'),
             State('Countries_data', 'data'),
             Input('Remove','n_clicks'),
             Input('Remove2','n_clicks'),
             Input('countries', 'value'),
             Input('Remove3','n_clicks'),
             Input('mark','n_clicks'),
             )

def update_account_details(hoverData,Regional_indicator,data,n_clicks,n_clicks2,country,n_clicks3,n_clicks4):


   
    if "Remove" == ctx.triggered_id:
        if len(data)==0:
           dff=pd.DataFrame(data=data).columns[:13]

      
        else:
           if  hoverData != {'points': [{'customdata': 'Finland'}]}:
              array=np.array(pd. DataFrame(data)['Country name'])
              mask = np.isin(array, hoverData['points'][0]['hovertext'])
              array = np.delete(array, mask)
              dff=df[df['Country name'].isin( array )][df.columns[:12]].round(2)
           
           else:
              array=np.array(pd. DataFrame(data)['Country name'])
              mask = np.isin(array, hoverData['points'][0]['customdata'])
              array = np.delete(array, mask)
              dff=df[df['Country name'].isin( array )][df.columns[:12]]
        
    elif "Remove2"==ctx.triggered_id:
         if len(data)==0:
            dff=pd.DataFrame(data=data)[df.columns[:12]]
           
      
         else:
            array=np.array(pd. DataFrame(data)['Country name'])
            array=array[:-1]
            dff=df[df['Country name'].isin( array )][df.columns[:12]]
      
    elif "Remove3"== ctx.triggered_id:
        if len(data)==0:
           dff=pd.DataFrame(data=data)[df.columns[:12]]
           
        else:
           array=np.array(pd. DataFrame(data)['Country name'])
           mask = np.isin(array, country)
           array = np.delete(array, mask)
           dff=df[df['Country name'].isin( array )][df.columns[:12]]
                  

    elif "mark" == ctx.triggered_id:
        if len(data)==0:
           array=np.array(pd. DataFrame(data))
           array = np.append(array, country)
           dff=df[df['Country name'].isin( array )][df.columns[:12]]
          
           
        else:
           df1=pd. DataFrame(data)
          
           df2=df[df['Country name'].isin( [country] )].round(2)
           dff=pd.concat([df1,df2])[df.columns[:12]]
          
           dff=dff.round(2)
          
        
    else:
         if len(data)==0:
             array=np.array(pd. DataFrame(data))
         else:
             array=np.unique(pd. DataFrame(data)['Country name'])


         if  hoverData != {'points': [{'customdata': 'Finland'}]}:
            array = np.append(array, hoverData['points'][0]['hovertext'])
            dff = df[df['Country name'].isin(array)][df.columns[:12]]
         else:
            array = np.append(array, hoverData['points'][0]['customdata'])
            dff = df[df['Country name'].isin(array)][df.columns[:12]]
         dff=dff.round(2) 
    columns = [{'name': col, 'id': col} for col in dff.columns]
    data = dff.to_dict(orient='records')

    


    
    return data


@callback(
             Output('Countries_table', 'data'),
             Output('Countries_table', 'columns'),
              Input('Countries_data', 'data')
   
             )

def update_account_details(data):
    dff=pd. DataFrame(data)
    columns = [{'name': col, 'id': col} for col in dff.columns]
    data = dff.to_dict(orient='records')
    return data,columns

    
if __name__ == '__main__':
 
    app.run(jupyter_mode="tab",debug=True)
 

Dash app running on http://127.0.0.1:8050/


<IPython.core.display.Javascript object>